In [ ]:
# !pip install optuna

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import gc, warnings, random, time, os

from pathlib import Path

from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import Adam, lr_scheduler
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from transformers import AutoModel, AutoTokenizer, AutoConfig
from transformers import get_cosine_schedule_with_warmup

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import seaborn as sns

import gc
gc.enable()

import optuna

### Folders and Dataframes

In [2]:
DATA_PATH = Path('/home/commonlit/data/')
assert DATA_PATH.exists()
MODELS_PATH = Path('/home/commonlit/models/')
if not MODELS_PATH.exists():
    os.mkdir(MODELS_PATH)
assert MODELS_PATH.exists()

In [3]:
train_df = pd.read_csv(DATA_PATH/'train-orig.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')
sample_df = pd.read_csv(DATA_PATH/'sample_submission.csv')

In [4]:
def remove_unnecessary(df):
    df.drop(df[df['target'] == 0].index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
remove_unnecessary(train_df)

In [5]:
train_df

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845
...,...,...,...,...,...,...
2828,25ca8f498,https://sites.ehe.osu.edu/beyondpenguins/files...,CC BY-SA 3.0,When you think of dinosaurs and where they liv...,1.711390,0.646900
2829,2c26db523,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,So what is a solid? Solids are usually hard be...,0.189476,0.535648
2830,cd19e2350,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,The second state of matter we will discuss is ...,0.255209,0.483866
2831,15e2e9e7a,https://en.wikibooks.org/wiki/Geometry_for_Ele...,CC BY-SA 3.0,Solids are shapes that you can actually touch....,-0.215279,0.514128


### Config and Seeding

In [6]:
class Config(): 
    NUM_FOLDS = 6
    NUM_EPOCHS = 3
    BATCH_SIZE = 16
    MAX_LEN = 248
    EVAL_SCHEDULE = [(0.50, 16), (0.49, 8), (0.48, 4), (0.47, 2), (-1., 1)]
    MODEL_PATH = 'xlnet-large-cased'
    TOKENIZER_PATH = 'xlnet-large-cased'
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    SEED = 1000
    NUM_WORKERS = 2
    MODEL_FOLDER = MODELS_PATH
    model_name = 'xlnet-large-cased'
    svm_kernels = ['rbf']
    svm_c = 5

cfg = Config()

In [7]:
if not cfg.MODEL_FOLDER.exists():
    os.mkdir(cfg.MODEL_FOLDER)

In [8]:
def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    os.environ["PYTHONHASHSEED"] = str(random_seed)

    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)

    torch.backends.cudnn.deterministic = True

### Dataset

In [9]:
def add_bins(train_df, num_bins):
    train_df.loc[:, 'bins'] = pd.cut(train_df['target'], bins=num_bins, labels=False)
    return num_bins

In [10]:
add_bins(train_df, cfg.NUM_FOLDS)

6

In [11]:
train_df.groupby(['bins'])['target'].agg(['count', 'mean'])

,count,mean
bins,,
0,122,-3.125765
1,441,-2.270279
2,784,-1.412150
3,886,-0.548095
4,494,0.289716
5,106,1.070237


In [2]:
tokenizer = AutoTokenizer.from_pretrained(cfg.TOKENIZER_PATH)

NameError: name 'AutoTokenizer' is not defined

In [1]:
dir(tokenizer)

NameError: name 'tokenizer' is not defined

In [13]:
class CommonLitDataset(Dataset):
    def __init__(self, df, tokenizer, inference_only=False):
        super().__init__()
        self.df, self.inference_only = df, inference_only
        self.text = df['excerpt'].tolist()
        self.bins = df['bins']
        if not inference_only:
            self.target = torch.tensor(df['target'].to_numpy(), dtype = torch.float32)
        
        self.encoded = tokenizer.batch_encode_plus(
            self.text,
            padding = 'max_length',
            max_length = cfg.MAX_LEN,
            truncation = True,
            return_attention_mask=True
        )
        self.tokenizer = tokenizer
        
    def __getitem__(self, index):        
        input_ids = torch.tensor(self.encoded['input_ids'][index])
        attention_mask = torch.tensor(self.encoded['attention_mask'][index])
        
        if self.inference_only:
            return {'input_ids': input_ids, 'attention_mask': attention_mask}
        else:
            target = self.target[index]
            return {'input_ids': input_ids, 'attention_mask': attention_mask, 'target': target}
    
    def __len__(self):
        return len(self.df)

In [14]:
sample_ds = CommonLitDataset(train_df, tokenizer)

### Model

In [15]:
class AttentionHead(nn.Module):
    
    def __init__(self, in_features, hidden_dim, num_targets):
        super().__init__()
        self.in_features = in_features
        
        self.hidden_layer = nn.Linear(in_features, hidden_dim)
        self.final_layer = nn.Linear(hidden_dim, num_targets)
        self.out_features = hidden_dim
        
    def forward(self, features):
        att = torch.tanh(self.hidden_layer(features))
        score = self.final_layer(att)
        attention_weights = torch.softmax(score, dim=1)
        return attention_weights

In [16]:
from transformers import AutoModelForSequenceClassification

class CommonLitModel(nn.Module):
    def __init__(self):
        super(CommonLitModel, self).__init__()
        config = AutoConfig.from_pretrained(cfg.MODEL_PATH)
        config.update({
            "output_hidden_states": True,
            "hidden_dropout_prob": 0.0,
            "layer_norm_eps": 1e-7
        })
        self.transformer_model = AutoModelForSequenceClassification.from_pretrained(cfg.MODEL_PATH, config=config)
        self.attention = AttentionHead(config.hidden_size, 512, 1)
        self.regressor = nn.Linear(config.hidden_size, 1)
    
    def forward(self, input_ids, attention_mask):
        hidden_states = self.transformer_model(input_ids=input_ids, attention_mask=attention_mask)['hidden_states']
        last_layer_hidden_states = hidden_states[-1]
        weights = self.attention(last_layer_hidden_states)
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1) 
        return self.regressor(context_vector), context_vector

In [17]:
sample_model = CommonLitModel()

Some weights of the model checkpoint at xlnet-large-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-large-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions

In [18]:
import re

for i, (name, param) in enumerate(sample_model.named_parameters()):
    if(name.find('layer') > -1):
        layer_name = re.sub(r'.+(layer\.\d+).+', r'\1', name)

In [19]:
for i, (name, param) in enumerate(sample_model.named_parameters()):
    print(i, name, param.size())

0 transformer_model.transformer.mask_emb torch.Size([1, 1, 1024])
1 transformer_model.transformer.word_embedding.weight torch.Size([32000, 1024])
2 transformer_model.transformer.layer.0.rel_attn.q torch.Size([1024, 16, 64])
3 transformer_model.transformer.layer.0.rel_attn.k torch.Size([1024, 16, 64])
4 transformer_model.transformer.layer.0.rel_attn.v torch.Size([1024, 16, 64])
5 transformer_model.transformer.layer.0.rel_attn.o torch.Size([1024, 16, 64])
6 transformer_model.transformer.layer.0.rel_attn.r torch.Size([1024, 16, 64])
7 transformer_model.transformer.layer.0.rel_attn.r_r_bias torch.Size([16, 64])
8 transformer_model.transformer.layer.0.rel_attn.r_s_bias torch.Size([16, 64])
9 transformer_model.transformer.layer.0.rel_attn.r_w_bias torch.Size([16, 64])
10 transformer_model.transformer.layer.0.rel_attn.seg_embed torch.Size([2, 16, 64])
11 transformer_model.transformer.layer.0.rel_attn.layer_norm.weight torch.Size([1024])
12 transformer_model.transformer.layer.0.rel_attn.layer_

In [20]:
sample_input_ids = torch.randint(0, 1000, [8, 248])
sample_attention_mask = torch.randint(0, 1000, [8, 248])

In [21]:
sample_model(sample_input_ids, sample_attention_mask)[1].shape

torch.Size([8, 1024])

In [22]:
torch.sum(torch.randn([8, 496, 768]), axis=1)

tensor([[-13.6484,  24.9908,  -8.6831,  ...,  -3.9337,  19.5884,   3.1343],
        [-16.2897, -26.9207, -18.0901,  ...,  -2.8086, -12.4488,  12.2931],
        [-30.4366, -16.2234,   9.5971,  ...,  29.0412, -13.2017,   1.4963],
        ...,
        [ 11.8891,  20.6609,  34.4373,  ...,  15.0036,  10.8892,  14.0213],
        [-20.6304,  21.4410,  22.3446,  ...,  -1.2921,  30.8943, -24.8172],
        [ -2.9513,   4.5531,  12.7362,  ...,   1.6763,   4.2718,  -8.4886]])

### Evaluation and Prediction

In [23]:
def eval_mse(model, data_loader):
    model.eval()
    mse_sum = 0
    mse_loss = nn.MSELoss(reduction='sum')
    
    with torch.no_grad():
        for batch_num, record in enumerate(data_loader):
            input_ids, attention_mask, target = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE), record['target'].to(cfg.DEVICE)
            pred, _ = model(input_ids, attention_mask)
            mse_sum += mse_loss(pred.flatten().cpu(), target.cpu())
            
    return mse_sum / len(data_loader.dataset)

In [24]:
def predict(model, data_loader):
    model.eval()
    result = []
    
    with torch.no_grad():
        for batch_num, record in tqdm(enumerate(data_loader), total=len(data_loader)):
            input_ids, attention_mask = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE)
            pred, _ = model(input_ids, attention_mask)
            result.extend(pred.flatten().to("cpu").tolist())
            
    return np.array(result)

In [25]:
sample_dl = DataLoader(sample_ds, shuffle=False, batch_size=16, num_workers=1)

### Optimizer and Sampler

In [26]:
5e-5 / 2.5, 5e-5 / 0.5, 5e-5

(2e-05, 0.0001, 5e-05)

In [27]:
def create_optimizer(model, base_lr=5e-5, last_lr=None):
    named_parameters = list(model.named_parameters())
    attention_param_start = 414
    regressor_param_start = 418
    roberta_parameters = named_parameters[:attention_param_start]
    attention_parameters = named_parameters[attention_param_start:regressor_param_start]
    regressor_parameters = named_parameters[regressor_param_start:]
    
    attention_group = [params for (name, params) in attention_parameters]
    regressor_group = [params for (name, params) in regressor_parameters]
    
    parameters = []
    if last_lr is not None:
        parameters.append({"params": attention_group, "lr": last_lr})
        parameters.append({"params": regressor_group, "lr": last_lr})
    else:
        parameters.append({"params": attention_group})
        parameters.append({"params": regressor_group})
        
    # Change on different models
    layer_low_threshold = 325
    layer_middle_threshold = 189
    
    for layer_num, (name, params) in enumerate(roberta_parameters):
        weight_decay = 0.0 if 'bias' in name else 0.01
        
        lr = base_lr / 2.5 # 2e-05
        if layer_num >= layer_middle_threshold:
            lr = base_lr / 0.5 # 1e-4
        elif layer_num >= layer_low_threshold:        
            lr = base_lr    
            
        parameters.append({"params": params,
                           "weight_decay": weight_decay,
                           "lr": lr})
        
    return AdamW(parameters)

In [28]:
sample_optimizer = create_optimizer(sample_model)

In [29]:
from torch.utils.data import Sampler,SequentialSampler,RandomSampler,SubsetRandomSampler
from collections import Counter

class WeightedSampler(Sampler):
    
    def __init__(self, dataset):
        
        self.indices = list(range(len(dataset)))
        self.num_samples = len(dataset)
        self.label_to_count = dict(Counter(dataset.bins))
        weights = [1/self.label_to_count[i] for i in dataset.bins]
        
        self.weights = torch.tensor(weights,dtype=torch.double)
        
    def __iter__(self):
        count = 0
        index = [self.indices[i] for i in torch.multinomial(self.weights, self.num_samples, replacement=True)]
        while count < self.num_samples:
            yield index[count]
            count += 1
    
    def __len__(self):
        return self.num_samples

### Training

In [30]:
def choose_eval_period(val_rmse):
    for rmse, period in cfg.EVAL_SCHEDULE:
        if val_rmse >= rmse:
            return period

In [31]:
def serialize_best(best_val_rmse, best_epoch, val_rmse, epoch, model, model_path):
    if not best_val_rmse or val_rmse < best_val_rmse:
        best_val_rmse = val_rmse
        best_epoch = epoch
        if not model_path.parent.exists():
            os.makedirs(model_path.parent)
        
        torch.save(model.state_dict(), model_path)
        print(f"New best_val_rmse: {best_val_rmse:0.4}")
    else:       
        print(f"Still best_val_rmse: {best_val_rmse:0.4}",
              f"(from epoch {best_epoch})")
    return best_epoch, best_val_rmse

In [39]:
class Trainer():
    def __init__(self, scaler, model, model_path, train_loader, val_loader, optimizer, scheduler=None, num_epochs=cfg.NUM_EPOCHS):
        self.scaler, self.model, self.model_path, self.train_loader, self.val_loader, self.optimizer, self.scheduler, self.num_epochs = (
            scaler, model, model_path, train_loader, val_loader, optimizer, scheduler, num_epochs
        )
            
    def train(self):
        self.model.train()
        
        mse_loss = nn.MSELoss(reduction='mean')
        
        best_val_rmse = None
        best_epoch = 0
        step = 0
        last_eval_step = 0
        eval_period = cfg.EVAL_SCHEDULE[0][1]    

        start = time.time()
        val_rmse_list = []
        
        tbar = tqdm(range(self.num_epochs), total=self.num_epochs)
        for epoch in tbar:
            tbar.set_description(f'Epoch: {epoch}')
            val_rmse = None
            for batch_num, record in enumerate(self.train_loader):
                input_ids, attention_mask, target = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE), record['target'].to(cfg.DEVICE)
                
                self.optimizer.zero_grad()
                
                # Casts operations to mixed precision
                with torch.cuda.amp.autocast():
                    pred, _ = self.model(input_ids, attention_mask)
                    mse = mse_loss(pred.flatten(), target)
                    
                self.scaler.scale(mse).backward()
                self.scaler.step(self.optimizer)
                self.scaler.update()
                
#                 mse.backward()
#                 self.optimizer.step()
                
                if self.scheduler:
                    self.scheduler.step()
                    
                if step >= last_eval_step + eval_period:
                    elapsed_seconds = time.time() - start
                    num_steps = step - last_eval_step
                    print(f"\n{num_steps} steps took {elapsed_seconds:0.3} seconds")
                    last_eval_step = step
                    
                    val_rmse = np.sqrt(eval_mse(self.model, self.val_loader))
                    print(f"Epoch: {epoch} batch_num: {batch_num}", f"val_rmse: {val_rmse:0.4} ", end='')
                    
                    eval_period = choose_eval_period(val_rmse)
                    best_epoch, best_val_rmse = serialize_best(best_val_rmse, best_epoch, val_rmse, epoch, self.model, self.model_path)
                    val_rmse_list.append(val_rmse)
                    start = time.time()
                # Finish early on condition
                if epoch > 0 and best_val_rmse > 0.6 or (len(val_rmse_list) > 5 and np.array(val_rmse_list).mean() > 1.0):
                    return best_val_rmse
                
                step += 1
        return best_val_rmse

In [40]:
kfold = KFold(n_splits=cfg.NUM_FOLDS, random_state=cfg.SEED, shuffle=True)
splits = list(kfold.split(train_df))

### Main Training

In [41]:
def train_fold(base_lr, last_lr, fold=0, epochs=3):
    
    print(f'##### Using fold {fold}')
    print(f'##### Using base_lr {base_lr} last_lr {last_lr}')
    
    model_path = cfg.MODEL_FOLDER/f"{cfg.model_name.replace('/', '_')}_{fold + 1}/model_{fold + 1}.pth"
    
    set_random_seed(cfg.SEED + fold)
    
    tokenizer = AutoTokenizer.from_pretrained(cfg.TOKENIZER_PATH)
    
    train_indices, val_indices = splits[fold]
    train_dataset = CommonLitDataset(train_df.loc[train_indices], tokenizer)    
    val_dataset = CommonLitDataset(train_df.loc[val_indices], tokenizer)
    
    train_loader = DataLoader(train_dataset, batch_size=cfg.BATCH_SIZE,
                              drop_last=False, shuffle=True, num_workers=cfg.NUM_WORKERS)    
    val_loader = DataLoader(val_dataset, batch_size=cfg.BATCH_SIZE,
                            drop_last=False, shuffle=False, num_workers=cfg.NUM_WORKERS)
    
    set_random_seed(cfg.SEED + fold)
    
    model = CommonLitModel().to(cfg.DEVICE)
    
    optimizer = create_optimizer(model, base_lr=base_lr, last_lr=last_lr)
    
    scheduler = get_cosine_schedule_with_warmup(optimizer,
                                                num_training_steps=cfg.NUM_EPOCHS * len(train_loader), 
                                                num_warmup_steps=50)
    
    scaler = torch.cuda.amp.GradScaler()
    
    trainer = Trainer(scaler, model, model_path, train_loader, val_loader, optimizer, scheduler=scheduler, num_epochs=epochs)
    rmse_val = trainer.train()
    tokenizer.save_pretrained(str(model_path.parent))
    
    return rmse_val

In [42]:
# Best results
# Fold 0: { 'base_lr': 0.0001190683694379101, 'last_lr': 0.00017987585986205585, 'epochs': 4 } Best value: 0.49271923303604126
# Fold 1: {'base_lr': 0.00012114635348406963, 'last_lr': 0.0005477206613438486, 'epochs': 4}. Best value:  0.45853328704833984
# Fold 2: {'base_lr': 5.24730490640746e-05, 'last_lr': 0.00020041362261812433, 'epochs': 4}   Best value:  0.49088865518569946
# Fold 3: {'base_lr': 6.108276630664184e-05, 'last_lr': 0.00011544056953737668, 'epochs': 4}. Best value:  0.4930591881275177
# Fold 4: {'base_lr': 0.0001717178883932075, 'last_lr': 0.00042448836147656634, 'epochs': 4}  Best value:  0.48955243825912476
# Fold 5: {'base_lr': 0.000135700916847811, 'last_lr': 0.0029640935672153, 'epochs': 4}.      Best value:  0.4688156247138977

In [43]:
lr_list = [
    {'base_lr': 0.0001190683694379101, 'last_lr': 0.00017987585986205585, 'epochs': 4 },
    {'base_lr': 0.00012114635348406963, 'last_lr': 0.0005477206613438486, 'epochs': 4},
    {'base_lr': 5.24730490640746e-05, 'last_lr': 0.00020041362261812433, 'epochs': 4},
    {'base_lr': 6.108276630664184e-05, 'last_lr': 0.00011544056953737668, 'epochs': 4},
    {'base_lr': 0.0001717178883932075, 'last_lr': 0.00042448836147656634, 'epochs': 4},
    {'base_lr': 0.000135700916847811, 'last_lr': 0.0029640935672153, 'epochs': 4}  
]

In [44]:
%%time

rmse_values = []
for i in range(len(list(splits))):
    fold = i
    lrs = lr_list[fold]
    rmse_val = train_fold(lrs['base_lr'], lrs['last_lr'], fold=fold, epochs=lrs['epochs'])
    print(f'Final RMSE: {rmse_val}')
    rmse_values.append(rmse_val)

##### Using fold 0
##### Using base_lr 0.0001190683694379101 last_lr 0.00017987585986205585


Some weights of the model checkpoint at xlnet-large-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-large-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions


16 steps took 7.48 seconds
Epoch: 0 batch_num: 16 val_rmse: 1.234 New best_val_rmse: 1.234

16 steps took 6.65 seconds
Epoch: 0 batch_num: 32 val_rmse: 1.083 New best_val_rmse: 1.083

16 steps took 6.79 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.9715 New best_val_rmse: 0.9715

16 steps took 6.87 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6267 New best_val_rmse: 0.6267

16 steps took 6.6 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.7796 Still best_val_rmse: 0.6267 (from epoch 0)

16 steps took 6.57 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5865 New best_val_rmse: 0.5865

16 steps took 6.63 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.6631 Still best_val_rmse: 0.5865 (from epoch 0)

16 steps took 6.57 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.6396 Still best_val_rmse: 0.5865 (from epoch 0)

16 steps took 6.57 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5969 Still best_val_rmse: 0.5865 (from epoch 0)

16 steps took 7.14 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.6296 Still best_val_rmse:

Some weights of the model checkpoint at xlnet-large-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-large-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions


16 steps took 7.74 seconds
Epoch: 0 batch_num: 16 val_rmse: 2.01 New best_val_rmse: 2.01

16 steps took 6.56 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.8928 New best_val_rmse: 0.8928

16 steps took 6.62 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.9454 Still best_val_rmse: 0.8928 (from epoch 0)

16 steps took 6.95 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.7085 New best_val_rmse: 0.7085

16 steps took 7.0 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6477 New best_val_rmse: 0.6477

16 steps took 7.02 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.7885 Still best_val_rmse: 0.6477 (from epoch 0)

16 steps took 6.56 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.6322 New best_val_rmse: 0.6322

16 steps took 6.6 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5733 New best_val_rmse: 0.5733

16 steps took 6.69 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5996 Still best_val_rmse: 0.5733 (from epoch 0)

16 steps took 7.09 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5363 New best_val_rmse: 0.5363

16 steps to

...


16 steps took 7.58 seconds
Epoch: 0 batch_num: 16 val_rmse: 1.048 New best_val_rmse: 1.048

16 steps took 6.59 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7279 New best_val_rmse: 0.7279

16 steps took 6.79 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7155 New best_val_rmse: 0.7155

16 steps took 6.71 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.7608 Still best_val_rmse: 0.7155 (from epoch 0)

16 steps took 6.57 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6977 New best_val_rmse: 0.6977

16 steps took 6.67 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5527 New best_val_rmse: 0.5527

16 steps took 6.77 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5965 Still best_val_rmse: 0.5527 (from epoch 0)

16 steps took 6.58 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5268 New best_val_rmse: 0.5268

16 steps took 6.7 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.57 Still best_val_rmse: 0.5268 (from epoch 0)

16 steps took 7.15 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5818 Still best_val_rmse: 0.5268 (from epo

Some weights of the model checkpoint at xlnet-large-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-large-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions


16 steps took 7.6 seconds
Epoch: 0 batch_num: 16 val_rmse: 1.608 New best_val_rmse: 1.608

16 steps took 6.61 seconds
Epoch: 0 batch_num: 32 val_rmse: 1.143 New best_val_rmse: 1.143

16 steps took 6.59 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.9745 New best_val_rmse: 0.9745

16 steps took 6.82 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.7911 New best_val_rmse: 0.7911

16 steps took 6.7 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6734 New best_val_rmse: 0.6734

16 steps took 6.81 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.6363 New best_val_rmse: 0.6363

16 steps took 6.64 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.6557 Still best_val_rmse: 0.6363 (from epoch 0)

16 steps took 6.59 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.6154 New best_val_rmse: 0.6154

16 steps took 6.59 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.597 New best_val_rmse: 0.597

16 steps took 7.17 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5882 New best_val_rmse: 0.5882

16 steps took 6.61 seconds
Epoch: 1 batch_num: 

Some weights of the model checkpoint at xlnet-large-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-large-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions


16 steps took 7.65 seconds
Epoch: 0 batch_num: 16 val_rmse: 1.176 New best_val_rmse: 1.176

16 steps took 6.59 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.8045 New best_val_rmse: 0.8045

16 steps took 6.6 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.6749 New best_val_rmse: 0.6749

16 steps took 6.69 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6764 Still best_val_rmse: 0.6749 (from epoch 0)

16 steps took 6.58 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6482 New best_val_rmse: 0.6482

16 steps took 6.59 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.7031 Still best_val_rmse: 0.6482 (from epoch 0)

16 steps took 6.64 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5976 New best_val_rmse: 0.5976

16 steps took 6.92 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.6539 Still best_val_rmse: 0.5976 (from epoch 0)

16 steps took 6.72 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5467 New best_val_rmse: 0.5467

16 steps took 7.09 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5525 Still best_val_rmse: 0.5467 (from e

In [45]:
f'mean RMSE values: {np.mean(np.array(rmse_values))}'

'mean RMSE values: 0.49054256081581116'

### Verify the model

In [46]:
from sklearn.svm import SVR
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm

In [47]:
cfg.model_offset = 0
cfg.model_limit = 6
cfg.n_folds = 5
cfg.svm_kernels = ['rbf']
cfg.svm_c = 5

In [48]:
num_bins = int(np.ceil(np.log2(len(train_df))))
train_df['bins'] = pd.cut(train_df['target'], bins=num_bins, labels=False)
bins = train_df['bins'].values

In [49]:
%%time

inference_models = []
for i in range(1, cfg.NUM_FOLDS + 1):
    print(f'Model {i}')
    inference_model = CommonLitModel()
    inference_model = inference_model.cuda()
    inference_model.load_state_dict(torch.load(str(MODELS_PATH/f"{cfg.model_name.replace('/', '_')}_{i}/model_{i}.pth")))
    inference_model.eval();
    inference_models.append(inference_model)

Model 1


Some weights of the model checkpoint at xlnet-large-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-large-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions

Model 2


Some weights of the model checkpoint at xlnet-large-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-large-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions

Model 3


...

Model 5


Some weights of the model checkpoint at xlnet-large-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-large-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions

Model 6


Some weights of the model checkpoint at xlnet-large-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-large-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions

CPU times: user 27.3 s, sys: 11.4 s, total: 38.6 s
Wall time: 35.4 s


In [51]:
from transformers import XLNetTokenizer

tokenizers = []
for i in range(1, cfg.NUM_FOLDS):
    tokenizer = XLNetTokenizer.from_pretrained(MODELS_PATH/f"{cfg.model_name.replace('/', '_')}_{i}")
    tokenizers.append(tokenizer)

In [52]:
def get_cls_embeddings(dl, transformer_model):
    cls_embeddings = []
    with torch.no_grad():
        for input_features in tqdm(dl, total=len(dl)):
            output, context_vector = transformer_model(input_features['input_ids'].cuda(), input_features['attention_mask'].cuda())
#             cls_embeddings.extend(output['last_hidden_state'][:,0,:].detach().cpu().numpy())
            embedding_out = context_vector.detach().cpu().numpy()
            cls_embeddings.extend(embedding_out)
    return np.array(cls_embeddings)

In [53]:
def rmse_score(X, y):
    return np.sqrt(mean_squared_error(X, y))

In [54]:
def convert_to_list(t):
    return t.flatten().long()

class CommonLitDataset(nn.Module):
    def __init__(self, text, test_id, tokenizer, max_len=128):
        self.excerpt = text
        self.test_id = test_id
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        return {'input_ids': convert_to_list(encode['input_ids']),
                'attention_mask': convert_to_list(encode['attention_mask']),
                'id': self.test_id[idx]}
    
    def __len__(self):
        return len(self.excerpt)

In [55]:
def create_dl(df, tokenizer):
    text = df['excerpt'].values
    ids = df['id'].values
    ds = CommonLitDataset(text, ids, tokenizer, max_len=cfg.MAX_LEN)
    return DataLoader(ds, 
                      batch_size = cfg.BATCH_SIZE,
                      shuffle=False,
                      num_workers = 1,
                      pin_memory=True,
                      drop_last=False
                     )

In [56]:
train_df = pd.read_csv(DATA_PATH/'train-orig.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')
remove_unnecessary(train_df)

In [57]:
train_target_mean = train_df['target'].mean()
train_target_std = train_df['target'].std()
train_df['normalized_target'] = (train_df['target'] - train_target_mean) / train_target_std

In [58]:
%%time

train_target = train_df['normalized_target'].values

def calc_mean(scores):
    return np.mean(np.array(scores), axis=0)

final_scores = []
final_rmse = []
kernel_rmse_score_mean = []
final_kernel_predictions_means = []
for j, (inference_model, tokenizer) in enumerate(zip(inference_models, tokenizers)):
    print('Model', j)
    test_dl = create_dl(test_df, tokenizer)
    train_dl = create_dl(train_df, tokenizer)
    transformer_model = inference_model
    transformer_model.cuda()
    X = get_cls_embeddings(train_dl, transformer_model)
    
    y = train_target
    X_test = get_cls_embeddings(test_dl, transformer_model)
    
    kfold = StratifiedKFold(n_splits=cfg.NUM_FOLDS)
    scores = []
    rmse_scores = []
    kernel_predictions_means = []
    for kernel in cfg.svm_kernels:
        print('Kernel', kernel)
        kernel_scores = []
        kernel_rmse_scores = []
        kernel_predictions = []
        for k, (train_idx, valid_idx) in enumerate(kfold.split(X, bins)):

            print('Fold', k, train_idx.shape, valid_idx.shape)
            model = SVR(C=cfg.svm_c, kernel=kernel, gamma='auto')

            X_train, y_train = X[train_idx], y[train_idx]
            X_valid, y_valid = X[valid_idx], y[valid_idx]
            model.fit(X_train, y_train)
            prediction = model.predict(X_valid)
            kernel_predictions.append(prediction)
            kernel_rmse_scores.append(rmse_score(prediction, y_valid))
            print('rmse_score', kernel_rmse_scores[k])
            kernel_scores.append(model.predict(X_test))
        kernel_predictions_means.append(np.array([np.mean(kp) for kp in kernel_predictions]).mean())
        scores.append(calc_mean(kernel_scores))
        kernel_rmse_score = calc_mean(kernel_rmse_scores)
        kernel_rmse_score_mean.append(kernel_rmse_score)
        rmse_scores.append(kernel_rmse_score)
    final_kernel_predictions_means.append(kernel_predictions_means)
    final_scores.append(calc_mean(scores))
    final_rmse.append(calc_mean(rmse_scores))
print('FINAL RMSE score', np.mean(np.array(final_rmse)))

Model 0



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.33515096550935586
Fold 1 (2361,) (472,)
rmse_score 0.3193098077707963
Fold 2 (2361,) (472,)
rmse_score 0.32251004947632655
Fold 3 (2361,) (472,)
rmse_score 0.29293254002942654
Fold 4 (2361,) (472,)
rmse_score 0.32323430296322003
Fold 5 (2361,) (472,)
rmse_score 0.3466767047877188
Model 1



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.3182430431461126
Fold 1 (2361,) (472,)
rmse_score 0.3219037882068193
Fold 2 (2361,) (472,)
rmse_score 0.2888435193030401
Fold 3 (2361,) (472,)
rmse_score 0.31096691853123826
Fold 4 (2361,) (472,)
rmse_score 0.30858202611891195
Fold 5 (2361,) (472,)
rmse_score 0.32937737896600716
Model 2



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.30032440012317796
Fold 1 (2361,) (472,)
rmse_score 0.31748911392174217
Fold 2 (2361,) (472,)
rmse_score 0.26298940173781604
Fold 3 (2361,) (472,)
rmse_score 0.29722142082890035
Fold 4 (2361,) (472,)
rmse_score 0.29362615459623664
Fold 5 (2361,) (472,)
rmse_score 0.31486126150890403
Model 3



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.32329676470865015
Fold 1 (2361,) (472,)
rmse_score 0.34047391247728676
Fold 2 (2361,) (472,)
rmse_score 0.3214505356250382
Fold 3 (2361,) (472,)
rmse_score 0.32061903880621
Fold 4 (2361,) (472,)
rmse_score 0.32456095571720045
Fold 5 (2361,) (472,)
rmse_score 0.3349042720639877
Model 4



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.3293199886529168
Fold 1 (2361,) (472,)
rmse_score 0.314861642129274
Fold 2 (2361,) (472,)
rmse_score 0.2981218258731394
Fold 3 (2361,) (472,)
rmse_score 0.3016225692878526
Fold 4 (2361,) (472,)
rmse_score 0.32915910446759183
Fold 5 (2361,) (472,)
rmse_score 0.3386687143116818
FINAL RMSE score 0.316043404054886
CPU times: user 8min 21s, sys: 8.83 s, total: 8min 30s
Wall time: 5min 2s


In [59]:
final_kernel_predictions_means

[[0.009306311708299965],
 [0.0010705579687654103],
 [-0.0016648509076259838],
 [0.008451955668970617],
 [0.004051537076226355]]

In [60]:
# (train_df['target'] - cfg.train_target_mean) / cfg.train_target_std
final_scores_normalized = np.array(final_scores) * train_target_std + train_target_mean

In [61]:
kernel_rmse_score_mean_array = np.array(kernel_rmse_score_mean)
kernel_rmse_score_mean_sum = np.sum(kernel_rmse_score_mean_array)
prop_losses = kernel_rmse_score_mean_array / kernel_rmse_score_mean_sum
prop_losses_sum = (1 - prop_losses).sum()
weights = (1 - prop_losses) / prop_losses_sum
weights

array([0.19885158, 0.20048368, 0.20289382, 0.19817944, 0.19959147])

In [62]:
def calc_mean(scores, weights=weights):
    return np.average(np.array(scores), weights=weights, axis=0)

In [63]:
target_mean = train_df['target'].mean()
final_scores_flat = calc_mean(final_scores_normalized).flatten()
final_scores_mean = final_scores_flat.mean()
target_mean, np.array(final_scores_normalized).mean()
# (-0.9579984513405823, -0.8029817438292849)

(-0.9596573929279916, -0.8581692301321396)

In [64]:
final_scores_flat

array([-0.22107183, -0.40934184, -0.51568938, -2.30189005, -1.91636142,
       -0.95218495,  0.3090395 ])

In [65]:
mean_diff = target_mean - final_scores_mean
mean_diff, mean_diff / len(final_scores)

(-0.10144311094582559, -0.020288622189165117)

In [66]:
sample_df['target'] = final_scores_flat + mean_diff
# sample_df['target'] = len(final_scores) / np.sum(1 / np.array(final_scores), axis=0) # harmonic mean
sample_df

,id,target
0,c0f722661,-0.322515
1,f0953f0a5,-0.510785
2,0df072751,-0.617132
3,04caf4e0c,-2.403333
4,0e63f8bea,-2.017805
5,12537fe78,-1.053628
6,965e592c0,0.207596


### Prepare Packaging

In [67]:
cfg.model_name

'xlnet-large-cased'

In [68]:
BEST_MODEL_FOLDER = MODELS_PATH/cfg.model_name/'best'
!rm -rf {BEST_MODEL_FOLDER}
!mkdir -p {BEST_MODEL_FOLDER}

In [69]:
BEST_MODEL_FOLDER

PosixPath('/home/commonlit/models/xlnet-large-cased/best')

In [70]:
cfg.NUM_FOLDS

6

In [71]:
bestmodels = [MODELS_PATH/f'{cfg.model_name}_{i + 1}' for i in range(0, cfg.NUM_FOLDS)]

In [72]:
bestmodels

[PosixPath('/home/commonlit/models/xlnet-large-cased_1'),
 PosixPath('/home/commonlit/models/xlnet-large-cased_2'),
 PosixPath('/home/commonlit/models/xlnet-large-cased_3'),
 PosixPath('/home/commonlit/models/xlnet-large-cased_4'),
 PosixPath('/home/commonlit/models/xlnet-large-cased_5'),
 PosixPath('/home/commonlit/models/xlnet-large-cased_6')]

In [74]:
from shutil import copyfile

def normalize_name(path_name):
    return path_name.replace('', '')

for i, best_model in enumerate(bestmodels):
    print(f'Processing {i}th model')
    i = i + 1
    best_model_file = f'{best_model}/model_{i}.pth'
    if Path(best_model_file).exists():
        copyfile(best_model_file, f'{BEST_MODEL_FOLDER}/{i}_pytorch_model.bin')
        tokenizer_path = Path(BEST_MODEL_FOLDER/f'tokenizer-{i}')
        tokenizer_path.mkdir(parents=True, exist_ok=True)
        assert tokenizer_path.exists()

        tokenizer_json = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/tokenizer_config.json'))
        assert tokenizer_json.exists(), f'{tokenizer_json} does not exist'
        copyfile(tokenizer_json, tokenizer_path/'tokenizer.json')

        vocab_txt = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/tokenizer.json'))
        assert vocab_txt.exists(), f'{vocab_txt} does not exist'
        copyfile(vocab_txt, tokenizer_path/'tokenizer.json')

        special_tokens = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/special_tokens_map.json'))
        assert special_tokens.exists()
        copyfile(special_tokens, tokenizer_path/'special_tokens_map')
        
        spiece_model = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/spiece.model'))
        assert spiece_model.exists()
        copyfile(spiece_model, tokenizer_path/'spiece.model')
    else:
        print(f'{best_model_file} is missing')

Processing 0th model
Processing 1th model
Processing 2th model
Processing 3th model
Processing 4th model
Processing 5th model


In [75]:
import shutil

shutil.make_archive(MODELS_PATH/cfg.model_name/'best_models', 'zip', BEST_MODEL_FOLDER)

'/home/commonlit/models/xlnet-large-cased/best_models.zip'

In [76]:
!ls {MODELS_PATH/cfg.model_name}

best  best_models.zip


In [77]:
!mv {MODELS_PATH}/{cfg.model_name}.yaml {MODELS_PATH/cfg.model_name}

mv: cannot stat '/home/commonlit/models/xlnet-large-cased.yaml': No such file or directory


In [78]:
transformer_model.transformer_model.save_pretrained(save_directory=f'{MODELS_PATH/cfg.model_name}/lm')

In [79]:
!du -h {MODELS_PATH/cfg.model_name}/*

2.1M	/home/commonlit/models/xlnet-large-cased/best/tokenizer-1
2.1M	/home/commonlit/models/xlnet-large-cased/best/tokenizer-2
2.1M	/home/commonlit/models/xlnet-large-cased/best/tokenizer-3
2.1M	/home/commonlit/models/xlnet-large-cased/best/tokenizer-4
2.1M	/home/commonlit/models/xlnet-large-cased/best/tokenizer-5
2.1M	/home/commonlit/models/xlnet-large-cased/best/tokenizer-6
8.2G	/home/commonlit/models/xlnet-large-cased/best
7.6G	/home/commonlit/models/xlnet-large-cased/best_models.zip
1.4G	/home/commonlit/models/xlnet-large-cased/lm


In [80]:
shutil.make_archive(MODELS_PATH/cfg.model_name/'lm', 'zip', f'{MODELS_PATH/cfg.model_name}/lm')

'/home/commonlit/models/xlnet-large-cased/lm.zip'

In [81]:
!kaggle datasets init -p {MODELS_PATH/cfg.model_name}

Data package template written to: /home/commonlit/models/xlnet-large-cased/dataset-metadata.json


In [82]:
dataset_json_path = Path(MODELS_PATH/cfg.model_name/'dataset-metadata.json')
assert dataset_json_path.exists()

In [83]:
!cat {str(dataset_json_path)}

{
  "title": "INSERT_TITLE_HERE",
  "id": "gilfernandes/INSERT_SLUG_HERE",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}

In [84]:
with open(dataset_json_path, 'r') as f:
    dataset_json = f.read()
    dataset_json = dataset_json.replace('INSERT_TITLE_HERE', f'commonlit-{cfg.model_name}').replace('INSERT_SLUG_HERE', f'commonlit-{cfg.model_name}')
    print(dataset_json)
with(open(dataset_json_path, 'w')) as f:
    f.write(dataset_json)

{
  "title": "commonlit-xlnet-large-cased",
  "id": "gilfernandes/commonlit-xlnet-large-cased",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}


In [85]:
!rm -rf {MODELS_PATH/cfg.model_name}/best
!rm -rf {MODELS_PATH/cfg.model_name}/lm

In [86]:
!kaggle datasets create -p {MODELS_PATH/cfg.model_name}

Starting upload for file best_models.zip
100%|██████████████████████████████████████| 7.52G/7.52G [12:42<00:00, 10.6MB/s]
Upload successful: best_models.zip (8GB)
Starting upload for file lm.zip
100%|██████████████████████████████████████| 1.25G/1.25G [02:04<00:00, 10.8MB/s]
Upload successful: lm.zip (1GB)
Your private Dataset is being created. Please check progress at /api/v1/datasets/status//gilfernandes/commonlit-xlnet-large-cased


In [ ]:
!kaggle datasets version -p {MODELS_PATH/cfg.model_name} -m "Version with spiece.model" -d

In [ ]:
state_dict = torch.load(str(MODELS_PATH/f'distilroberta-0/checkpoint-105/pytorch_model.bin'))

In [ ]:
loaded_model = CommonLitModel()

In [ ]:
loaded_model.load_state_dict(state_dict)